**Notebook to investigate the Young Stellar Object R CrA in Corona Australis with ALMA and other archival data** \
More information on querying the ALMA archive programatically: \
https://almascience.eso.org/alma-data/archive/archive-notebooks \
Documentation for *astroquery*: \
https://astroquery.readthedocs.io/

In [55]:
from astroquery.simbad import Simbad
from astroquery.alma import Alma
from astropy import coordinates
import astropy.units as u
import numpy as np

In [2]:
# Query Simbad by source name and retrieve coordinates, likely from YSO in the optical or near-IR
result_table = Simbad.query_object("R CrA")
result_table.pprint()
RA_Simbad = result_table['RA']
Dec_Simbad = result_table['DEC']

MAIN_ID        RA           DEC      RA_PREC DEC_PREC COO_ERR_MAJA COO_ERR_MINA COO_ERR_ANGLE COO_QUAL COO_WAVELENGTH     COO_BIBCODE     SCRIPT_NUMBER_ID
            "h:m:s"       "d:m:s"                         mas          mas           deg                                                                  
-------- ------------- ------------- ------- -------- ------------ ------------ ------------- -------- -------------- ------------------- ----------------
V* R CrA 19 01 53.6764 -36 57 08.299      14       14        0.435        0.417            90        A              O 2020yCat.1350....0G                1


In [3]:
# Also, define coordinates of YSO from VLA data. 
# Reference: Liu, Galván-Madrid, Forbrich et al. (2014)
# https://ui.adsabs.harvard.edu/abs/2014ApJ...780..155L/abstract 
RA_VLA = "19h01m53.686s"
Dec_VLA = "-36d57m08.0s"
c = coordinates.SkyCoord(RA_VLA, Dec_VLA, frame='icrs')
r = 1 * u.arcsec
result_table = Simbad.query_region(c, radius=r)
result_table.pprint()

MAIN_ID        RA           DEC      RA_PREC DEC_PREC COO_ERR_MAJA COO_ERR_MINA COO_ERR_ANGLE COO_QUAL COO_WAVELENGTH     COO_BIBCODE     SCRIPT_NUMBER_ID
            "h:m:s"       "d:m:s"                         mas          mas           deg                                                                  
-------- ------------- ------------- ------- -------- ------------ ------------ ------------- -------- -------------- ------------------- ----------------
V* R CrA 19 01 53.6764 -36 57 08.299      14       14        0.435        0.417            90        A              O 2020yCat.1350....0G                1


In [4]:
# Start digging in the ALMA archive, first check what we can do: 
Alma.help()
#Instantiate object for query: 
alma = Alma()


Most common ALMA query keywords are listed below. These keywords are part of the ALMA ObsCore model, an IVOA standard for metadata representation (3rd column). They were also present in original ALMA Web form and, for backwards compatibility can be accessed with their old names (2nd column).
More elaborate queries on the ObsCore model are possible with `query_sia` or `query_tap` methods
  Description                       Original ALMA keyword               ObsCore keyword                    
-------------------------------------------------------------------------------------------------------

Position
  Source name (astropy Resolver)    source_name_resolver                SkyCoord.from_name                 
  Source name (ALMA)                source_name_alma                    target_name                        
  RA Dec (Sexagesimal)              ra_dec                              s_ra, s_dec                        
  Galactic (Degrees)                galactic                   

In [93]:
# Define parameters for data query
# help(alma_data.query) 
band_list = [3]
spatial_resolution = 2.0*u.arcsec
search_radius = 65*u.arcsec

In [94]:
# Make data query: 
alma_data = alma.query_region(c, search_radius, band_list=band_list).to_pandas()
print(alma_data)
#print(len(alma_data.colnames))
#print(alma_data.colnames)
alma_data.to_csv('test_query_region.csv')

              obs_publisher_did obs_collection facility_name instrument_name  \
0   ADS/JAO.ALMA#2019.1.01074.S           ALMA           JAO            ALMA   
1   ADS/JAO.ALMA#2019.1.01074.S           ALMA           JAO            ALMA   
2   ADS/JAO.ALMA#2019.1.01074.S           ALMA           JAO            ALMA   
3   ADS/JAO.ALMA#2019.1.01074.S           ALMA           JAO            ALMA   
4   ADS/JAO.ALMA#2018.1.01205.L           ALMA           JAO            ALMA   
5   ADS/JAO.ALMA#2018.1.01205.L           ALMA           JAO            ALMA   
6   ADS/JAO.ALMA#2018.1.01205.L           ALMA           JAO            ALMA   
7   ADS/JAO.ALMA#2018.1.01205.L           ALMA           JAO            ALMA   
8   ADS/JAO.ALMA#2018.1.01205.L           ALMA           JAO            ALMA   
9   ADS/JAO.ALMA#2018.1.01205.L           ALMA           JAO            ALMA   
10  ADS/JAO.ALMA#2018.1.01205.L           ALMA           JAO            ALMA   
11  ADS/JAO.ALMA#2018.1.01205.L         

In [95]:
# Make a new, more refined query using TAP
# The format is a bit annoying but this is the most flexible way to query

alma_data = alma.query_tap(f"select * from ivoa.obscore where intersects(circle('ICRS',{c.ra.degree},"
                           f"{c.dec.degree},{search_radius.to('deg').value}),s_region)=1 and " 
                           f"band_list={band_list[0]} and spatial_resolution<={spatial_resolution.value}"
                          ).to_table().to_pandas()
print(alma_data)
alma_data.to_csv("test_query_tap.csv")

              obs_publisher_did obs_collection facility_name instrument_name  \
0   ADS/JAO.ALMA#2019.1.01074.S           ALMA           JAO            ALMA   
1   ADS/JAO.ALMA#2019.1.01074.S           ALMA           JAO            ALMA   
2   ADS/JAO.ALMA#2019.1.01074.S           ALMA           JAO            ALMA   
3   ADS/JAO.ALMA#2019.1.01074.S           ALMA           JAO            ALMA   
4   ADS/JAO.ALMA#2018.1.01205.L           ALMA           JAO            ALMA   
5   ADS/JAO.ALMA#2018.1.01205.L           ALMA           JAO            ALMA   
6   ADS/JAO.ALMA#2018.1.01205.L           ALMA           JAO            ALMA   
7   ADS/JAO.ALMA#2018.1.01205.L           ALMA           JAO            ALMA   
8   ADS/JAO.ALMA#2018.1.01205.L           ALMA           JAO            ALMA   
9   ADS/JAO.ALMA#2018.1.01205.L           ALMA           JAO            ALMA   
10  ADS/JAO.ALMA#2018.1.01205.L           ALMA           JAO            ALMA   
11  ADS/JAO.ALMA#2018.1.01205.L         

In [116]:
# Preparation for download 
alma.cache_location = '/home/roberto/YSOs/RCrA/'
uids = np.unique(alma_data['member_ous_uid'])
print(uids)
link_list = alma.get_data_info(uids, expand_tarfiles=True)
print(link_list)

['uid://A001/X1354/X240' 'uid://A001/X1354/X242' 'uid://A001/X1465/X1aca'
 'uid://A001/X2d1f/Xb26']
                        ID                                                                      access_url                                              service_def ... content_qualifier link_auth link_authorized
                                                                                                                                                                    ...                                            
-------------------------------------------------- ---------------------------------------------------------------------------------------------------- ----------- ... ----------------- --------- ---------------
                             uid://A001/X1354/X240                      https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.README.txt             ...                       false            True
                             uid://A001/X1354/X240  

In [120]:
# First, download only the QA2 FITS images and README files
fits_urls = [url for url in link_list['access_url'] if 'CrA' in url and '.pbcor.fits' in url and 
                'tt1' not in url]
print(len(fits_urls))
print(*fits_urls, sep='\n')

129
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw25.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw25.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw25_27_29_31_33_35_37.cont.I.tt0.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw27.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw27.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw29.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw29.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw31.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw31.mfs.I.pbcor.fi

In [113]:
# First, download only the QA2 FITS images and README files
fits_urls = []
print('######################################################################')
for mous in range(len(alma_data['member_ous_uid'])):
    uid_url_table = alma.get_data_info(alma_data['member_ous_uid'][mous], expand_tarfiles=True)
    #print(uid_url_table[0:5])
    print('------------------------------------------------------------------')
    # Broad search of fits files
    #fits_urls = [url for url in uid_url_table['access_url'] if '.fits' in url]
    # If you only want the fits files of the target source
    new_urls = [url for url in uid_url_table['access_url'] if 'CrA' in url and '.pbcor.fits' in url and 
                'tt1' not in url]
    print(len(new_urls))
    print(*new_urls, sep='\n')
    print('------------------------------------------------------------------')
    # Only cubes ? 
    #cube_urls = [url for url in uid_url_table['access_url'] if 'cube' in url]
    #print(*cube_urls, sep='\n')
    fits_urls.append(new_urls)
    #
    #filelist = alma.download_files(fits_urls[:5])


######################################################################
------------------------------------------------------------------
6
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1465_X1aca.RCrA_IRS7B_sci.spw21.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1465_X1aca.RCrA_IRS7B_sci.spw21_23_25_27.cont.I.tt0.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1465_X1aca.RCrA_IRS7B_sci.spw23.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1465_X1aca.RCrA_IRS7B_sci.spw23.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1465_X1aca.RCrA_IRS7B_sci.spw25.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1465_X1aca.RCrA_IRS7B_sci.spw27.mfs.I.pbcor.fits
------------------------------------------------------------------
------------------------------------------------------------------
6
https://almascience.nrao.edu/dataPortal/member.uid___A001_X14

------------------------------------------------------------------
15
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X242.R_CrA_IRS7B_sci.spw25.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X242.R_CrA_IRS7B_sci.spw25.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X242.R_CrA_IRS7B_sci.spw25_27_29_31_33_35_37.cont.I.tt0.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X242.R_CrA_IRS7B_sci.spw27.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X242.R_CrA_IRS7B_sci.spw27.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X242.R_CrA_IRS7B_sci.spw29.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X242.R_CrA_IRS7B_sci.spw29.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X242.R_CrA_IRS7B_sci.spw31.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal

------------------------------------------------------------------
15
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw25.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw25.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw25_27_29_31_33_35_37.cont.I.tt0.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw27.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw27.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw29.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw29.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw31.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal

------------------------------------------------------------------
15
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw25.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw25.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw25_27_29_31_33_35_37.cont.I.tt0.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw27.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw27.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw29.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw29.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X1354_X240.R_CrA_IRS7B_sci.spw31.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal

------------------------------------------------------------------
93
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23_25_27_29_31_33_35_37_39_41_43_45_47_49_51.cont.I.tt0.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw25.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw25.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw27.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw27.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw29.cube.I.pbcor.fits
https://almasci

...

------------------------------------------------------------------
93
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23_25_27_29_31_33_35_37_39_41_43_45_47_49_51.cont.I.tt0.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw25.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw25.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw27.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw27.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw29.cube.I.pbcor.fits
https://almasci

------------------------------------------------------------------
93
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23_25_27_29_31_33_35_37_39_41_43_45_47_49_51.cont.I.tt0.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw25.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw25.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw27.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw27.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw29.cube.I.pbcor.fits
https://almasci

------------------------------------------------------------------
93
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23_25_27_29_31_33_35_37_39_41_43_45_47_49_51.cont.I.tt0.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw25.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw25.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw27.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw27.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw29.cube.I.pbcor.fits
https://almasci

------------------------------------------------------------------
93
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23_25_27_29_31_33_35_37_39_41_43_45_47_49_51.cont.I.tt0.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw25.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw25.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw27.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw27.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw29.cube.I.pbcor.fits
https://almasci

------------------------------------------------------------------
93
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw23_25_27_29_31_33_35_37_39_41_43_45_47_49_51.cont.I.tt0.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw25.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw25.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw27.cube.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw27.mfs.I.pbcor.fits
https://almascience.nrao.edu/dataPortal/member.uid___A001_X2d1f_Xb26.RCrA_IRAS32_sci.spw29.cube.I.pbcor.fits
https://almasci

In [109]:
len(alma_data['member_ous_uid'])

48

In [112]:
uid_url_table.add_row

AttributeError: 'Table' object has no attribute 'append'